In [13]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv('./20180627_ballotimage.txt', sep=" ", header=None)
print(data.head(15))

                                                0
0   000002000001288600000010020000009001000018100
1   000002000001288600000010020000009002000018500
2   000002000001288600000010020000009003000018000
3   000002000001288700000010020000009001000018100
4   000002000001288700000010020000009002000018300
5   000002000001288700000010020000009003000018200
6   000002000001288800000010020000009001000018100
7   000002000001288800000010020000009002000017900
8   000002000001288800000010020000009003000018300
9   000002000001288900000010020000009001000018100
10  000002000001288900000010020000009002000017900
11  000002000001288900000010020000009003000018300
12  000002000001289000000010020000009001000017900
13  000002000001289000000010020000009002000018100
14  000002000001289000000010020000009003000018000


In [15]:
ballot = data.iloc[:, 0].astype(str)
print(type(ballot.str[0:4]))


<class 'pandas.core.series.Series'>


In [26]:
ballot_data = pd.DataFrame({
    'contest_id': ballot.str[0:7],
    'voter_number': ballot.str[7:16],
    'serial_number': ballot.str[16:23],
    'tally_type': ballot.str[23: 26],
    'precinct': ballot.str[26: 33],
    'rcv': ballot.str[33: 36],
    'candidate_number': ballot.str[36: 43],
    'over': ballot.str[43],
    'under': ballot.str[44]
})
print(ballot_data.head())

  contest_id voter_number serial_number tally_type precinct  rcv  \
0    0000020    000012886       0000001        002  0000009  001   
1    0000020    000012886       0000001        002  0000009  002   
2    0000020    000012886       0000001        002  0000009  003   
3    0000020    000012887       0000001        002  0000009  001   
4    0000020    000012887       0000001        002  0000009  002   

  candidate_number over under  
0          0000181    0     0  
1          0000185    0     0  
2          0000180    0     0  
3          0000181    0     0  
4          0000183    0     0  


In [17]:
ballot_data = pd.pivot_table(ballot_data, values='candidate_number', columns="rcv", index="serial_number", aggfunc="first")

print(ballot_data.head())

rcv                001      002      003
serial_number                           
0000001        0000181  0000185  0000180
0000002        0000179  0000179  0000179
0000003        0000180  0000179  0000185
0000004        0000185  0000180  0000179
0000005        0000180  0000179  0000178


In [21]:
voter_guide = pd.DataFrame({
    'alice': {'sup_1': '0000180', 'sup_2': '0000179', 'sup_3': '0000000'},
    'guardian': {'sup_1': '0000181', 'sup_2': '0000180', 'sup_3': '0000000'},
    'league': {'sup_1': '0000181', 'sup_2': '0000180', 'sup_3': '0000184'},
    'broke': {'sup_1': '0000181', 'sup_2': '0000180', 'sup_3': '0000184'},
    'berniecrats': {'sup_1': '0000181', 'sup_2': '0000184', 'sup_3': '0000180'}
}).T


#candidate codes
# Mark Leno = 0000180
# London Breed = 0000179
# Amy Farah Weiss = 0000184
# Jane Kim = 0000181
# None?????????? = 0000000

guide_tally = pd.DataFrame(columns=['tally'])
guide_tally.index.name = 'guide'

for index, row in voter_guide.iterrows():
  s = ballot_data[(ballot_data['001'] == row['sup_1']) & (ballot_data['002'] == row['sup_2']) & (ballot_data['003'] == row['sup_3'])]
  guide_tally.loc[index] = len(s.index)

print(guide_tally.head())


            tally
guide            
alice           3
guardian       18
league          9
broke           9
berniecrats     0
